# **"Guess the year" miniproject III**
**(Baglan AITU)**

In [ ]:
# Required libraries

import pandas as pd
import torch as t
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [ ]:
# Loading the dataset
data = pd.read_csv("/content/YearPredictionMSD.csv")
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,-2.46783,3.32136,-2.31521,10.20556,611.10913,951.08960,698.11428,408.98485,383.70912,326.51512,238.11327,251.42414,187.17351,100.42652,179.19498,-8.41558,-317.87038,95.86266,48.10259,-95.66303,-18.06215,1.96984,34.42438,11.72670,1.36790,7.79444,-0.36994,-133.67852,-83.26165,-37.29765,...,-25.38187,-3.90772,13.29258,41.55060,-7.26272,-21.00863,105.50848,64.29856,26.08481,-44.59110,-8.30657,7.93706,-10.73660,-95.44766,-82.03307,-35.59194,4.69525,70.95626,28.09139,6.02015,-37.13767,-41.12450,-8.40816,7.19877,-8.60176,-5.90857,-12.32437,14.68734,-54.32125,40.14786,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,2.21920,0.34006,44.38997,2056.93836,605.40696,457.41175,777.15347,415.64880,746.47775,366.45320,317.82946,273.07917,141.75921,317.35269,19.48271,-65.25496,162.75145,135.00765,-96.28436,-86.87955,17.38087,45.90742,32.49908,-32.85429,45.10830,26.84939,-302.57328,-41.71932,-138.85034,...,28.55107,1.52298,70.99515,-43.63073,-42.55014,129.82848,79.95420,-87.14554,-45.75446,-65.82100,-43.90031,-19.45705,12.59163,-407.64130,42.91189,12.15850,-88.37882,42.25246,46.49209,-30.17747,45.98495,130.47892,13.88281,-4.00055,17.85965,-18.32138,-87.99109,14.37524,-22.70119,-58.81266,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,2.73553,0.82804,7.46586,699.54544,1016.00954,594.06748,355.73663,507.39931,387.69910,287.15347,112.37152,161.68928,144.14353,199.29693,-4.24359,-297.00587,-148.36392,-7.94726,-18.71630,12.77542,-25.37725,9.71410,0.13843,26.79723,6.30760,28.70107,-74.89005,-289.19553,-166.26089,...,18.50939,16.97216,24.26629,-10.50788,-8.68412,54.75759,194.74034,7.95966,-18.22685,0.06463,-2.63069,26.02561,1.75729,-262.36917,-233.60089,-2.50502,-12.14279,81.37617,2.07554,-1.82381,183.65292,22.64797,-39.98887,43.37381,-31.56737,-4.88840,-36.53213,-23.94662,-84.19275,66.00518,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,6.63016,-3.35142,37.64085,2174.08189,697.43346,459.24587,742.78961,229.30783,387.89697,249.06662,245.89870,176.20527,98.82222,150.97286,78.49057,-62.00282,43.49659,-96.42719,-108.96608,14.22854,14.54178,-23.55608,-39.36953,-43.59209,20.83714,35.63919,-181.34947,-93.66614,-90.55616,...,4.56917,-37.32280,4.15159,12.24315,35.02697,-178.89573,82.46573,-20.49425,101.78577,-19.77808,-21.52657,3.36303,-11.63176,51.55411,-50.57576,-28.14755,-83.15795,-7.35260,-22.11505,1.18279,-122.70467,150.57360,24.37468,41.19821,-37.04318,-28.72986,162.19614,22.18309,-8.63509,85.23416,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,1.60923,2.19223,47.32082,894.28471,809.86615,318.78559,435.04497,341.61467,334.30734,322.99589,190.61921,235.84715,96.89517,210.58870,5.60463,-199.63958,204.85812,-77.17695,-65.79741,-6.95097,-12.15262,-3.85410,20.68990,-20.30480,37.15045,11.20673,-124.09519,-295.98542,-33.31169,...,45.25506,10.42226,27.88782,-17.12676,-31.54772,-76.86293,41.17343,-138.32535,-53.96905,-21.30266,-24.87362,-2.46595,-4.05003,-56.51161,-34.56445,-5.07092,-47.75605,64.81513,-97.42948,-12.59418,55.23699,28.85657,54.53513,-31.97077,20.03279,-8.07892,-55.12617,26.58961,-10.27183,-30.64232,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.6

In [ ]:
# Check for categorical features

cols = data.columns
num_cols = data._get_numeric_data().columns
categorical=list(set(cols) - set(num_cols))
print(categorical)

[]


In [ ]:
# Check for missing values

data.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
86    0
87    0
88    0
89    0
90    0
Length: 91, dtype: int64

In [ ]:
# Checking fot Outliers

outliers = {}
for i in range(data.shape[1]):
  min_t = data[data.columns[i]].mean()- (3 *
data[data.columns[i]].std())
  max_t = data[data.columns[i]].mean()+ (3 *
data[data.columns[i]].std())
  count = 0
  for j in data[data.columns[i]]:
    if j < min_t or j > max_t:
      count += 1
  percentage = count / data.shape[0]
  outliers[data.columns[i]] = "%.3f" % percentage
outliers

{'0': '0.019',
 '1': '0.010',
 '10': '0.005',
 '11': '0.010',
 '12': '0.010',
 '13': '0.015',
 '14': '0.017',
 '15': '0.016',
 '16': '0.016',
 '17': '0.014',
 '18': '0.016',
 '19': '0.013',
 '2': '0.011',
 '20': '0.016',
 '21': '0.015',
 '22': '0.013',
 '23': '0.015',
 '24': '0.013',
 '25': '0.018',
 '26': '0.017',
 '27': '0.017',
 '28': '0.018',
 '29': '0.017',
 '3': '0.011',
 '30': '0.016',
 '31': '0.015',
 '32': '0.017',
 '33': '0.016',
 '34': '0.015',
 '35': '0.016',
 '36': '0.017',
 '37': '0.017',
 '38': '0.016',
 '39': '0.015',
 '4': '0.015',
 '40': '0.017',
 '41': '0.017',
 '42': '0.015',
 '43': '0.014',
 '44': '0.016',
 '45': '0.015',
 '46': '0.017',
 '47': '0.017',
 '48': '0.017',
 '49': '0.015',
 '5': '0.008',
 '50': '0.016',
 '51': '0.015',
 '52': '0.015',
 '53': '0.015',
 '54': '0.016',
 '55': '0.018',
 '56': '0.017',
 '57': '0.016',
 '58': '0.013',
 '59': '0.016',
 '6': '0.010',
 '60': '0.016',
 '61': '0.015',
 '62': '0.016',
 '63': '0.017',
 '64': '0.017',
 '65': '0.016',

In [ ]:
# Data Rescaling

X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
X = (X - X.mean()) / X.std() # -average /std
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
0,1.082657,0.382437,1.841985,0.459652,-0.480074,-0.282606,-1.590785,-1.300854,0.378336,-0.683719,0.791667,-0.579058,-1.042779,-1.038170,-0.802033,-0.738707,-1.058872,-0.864608,-0.886254,-0.892847,-0.664183,-0.827553,-1.015349,-0.743670,-0.432437,-0.508796,0.269947,0.335583,-0.706657,-0.470577,-0.034705,0.135786,0.037951,0.095819,0.183362,-0.680166,-0.206765,-0.453855,0.590499,0.247393,...,-0.339102,-0.432580,0.022594,0.476840,0.043861,-0.316078,0.006113,0.243596,-0.069088,-0.106208,-0.197799,0.160338,-0.283001,0.144776,-0.379467,-0.287353,0.020455,0.376367,-0.041179,0.188516,-0.104213,-0.213180,0.117711,0.302813,0.204880,-0.021678,0.048260,-0.121904,0.250354,0.098260,-0.086005,0.099339,0.148291,-0.255625,0.040944,-0.362616,0.524542,-0.467668,-0.247579,0.036872
1,0.880874,0.321953,1.763666,0.717085,-0.165507,-1.188896,0.777905,0.122576,1.420531,0.401198,0.541262,-0.259082,0.495454,-0.203688,-1.079208,-0.955842,-0.272781,-0.809075,0.460392,-0.478534,-0.347118,-0.306398,-0.793396,0.169038,-0.200014,-0.157887,0.394176,0.733171,-0.710525,-0.980185,0.121372,0.296398,0.324097,-0.560564,1.070095,-0.423911,-0.609402,-0.361353,0.204130,0.892164,...,0.231116,-0.359945,0.831294,-0.549018,-0.570235,0.229851,-0.076646,-0.313931,-0.505686,-0.255144,-0.787143,-0.399109,0.342936,-0.885840,0.179196,0.087017,-0.915253,0.128522,0.137589,-0.812482,0.222566,0.556176,0.253343,0.124362,0.612455,-0.493883,-0.238847,-0.124105,0.405375,-0.712892,-0.316635,0.301448,-0.063611,0.031855,-0.655124,-0.443921,0.536517,0.573191,0.209887,1.155171
2,1.251484,0.588929,1.350579,0.745944,0.000857,-0.703401,-0.066747,-0.057380,1.163637,-0.090081,0.658570,-0.200277,-1.166060,-0.987128,-0.749978,-0.832567,-1.172565,-0.649550,-0.690063,-0.734533,-1.328116,-0.982155,-0.780592,-0.610871,-0.397680,-0.479813,-0.183643,0.079156,-0.227636,-0.242218,-0.311664,-0.209834,-0.121681,0.583554,0.148029,-0.406479,-0.066616,-0.912409,0.099843,-0.051943,...,0.124948,-0.153315,0.176390,-0.150112,0.019125,-0.041855,0.295097,0.036190,-0.338389,0.207068,-0.103821,0.529745,0.052231,-0.406266,-1.057168,-0.027947,-0.148824,0.466339,-0.293930,-0.028399,0.763778,0.072733,-0.074444,0.879232,-0.148851,0.017132,-0.043593,-0.394428,0.103906,0.310206,-0.399185,0.567666,-0.749508,-0.301984,-0.034072,0.227059,-0.528413,-0.335333,0.109957,-0.095865
3,0.800148,-0.082240,0.794774,0.081829,0.336246,-1.295366,0.517369,-1.062869,-0.029679,-1.282306,1.543411,-0.703927,0.191757,-0.136077,-1.005419,-0.954187,-0.346152,-1.133063,-0.689428,-0.857487,-0.690565,-0.894093,-1.023964,-0.930113,0.291586,-0.153369,0.172690,-0.325638,-0.789473,-0.231457,0.092619,-0.675181,-0.665915,-0.766515,0.493311,-0.341160,-0.320410,-0.477023,0.387871,-0.117401,...,-0.022438,-0.879503,-0.105516,0.123884,0.779816,-0.887524,-0.068512,-0.068560,0.390980,0.067864,-0.416689,0.066926,-0.307020,0.630060,-0.238813,-0.228988,-0.862765,-0.299797,-0.528948,0.054745,-0.440601,0.646267,0.317182,0.844566,-0.233192,-0.889840,0.710452,-0.069029,0.474336,0.467821,0.590596,-0.583396,-0.472129,-0.904164,-0.820141,0.577357,-0.282033,0.412329,0.961849,0.789313
4,1.253660,0.794806,1.671781,0.442438,-0.411071,-0.569426,-0.712128,-0.941459,0.836414,-0.160630,0.402680,-0.035885,0.627337,-0.874731,-0.915268,-1.080894,-1.003230,-0.937797,-0.861268,-0.618825,-0.954508,-0.532270,-1.034313,-0.536274,-0.315634,-0.344561,0.472378,-0.237568,-0.520733,-0.388296,-0.177731,-0.399611,0.161422,-0.319864,0.880984,-0.571178,-0.183919,-0.927528,0.605664,0.025919,...,0.407722,-0.240919,0.227146,-0.229825,-0.378764,-0.518233,-0.202240,-0.502345,-0.555610,0.057169,-0.472108,-0.052114,-0.103589,0.273312,-0.167222,-0.048064,-0.506857,0.323341,-1.260647,-0.326239,0.258938,0.100568,0.500696,-0.321321,0.645927,-0.104242,-0.114145,-0.037945,0.466312,

In [ ]:
# Split data

X.shape
train_end = int(len(X) * 0.8)
dev_end = int(len(X) * 0.9)

X_shuffle = X.sample(frac=1, random_state=0)
Y_shuffle = Y.sample(frac=1, random_state=0)
x_train = X_shuffle.iloc[:train_end,:]
y_train = Y_shuffle.iloc[:train_end]

x_dev = X_shuffle.iloc[train_end:dev_end,:]
y_dev = Y_shuffle.iloc[train_end:dev_end]
x_test = X_shuffle.iloc[dev_end:,:]
y_test = Y_shuffle.iloc[dev_end:]
print(x_train.shape, y_train.shape)
print(x_dev.shape, y_dev.shape)
print(x_test.shape, y_test.shape)

(40000, 90) (40000,)
(5000, 90) (5000,)
(5000, 90) (5000,)


In [ ]:
# Transform variables (be careful with cells, otherwise you will overwrite and obtain errors)

x_train = t.tensor(x_train.values).float()
y_train = t.tensor(y_train.values).float()
x_dev = t.tensor(x_dev.values).float()
y_dev = t.tensor(y_dev.values).float()
x_test = t.tensor(x_test.values).float()
y_test = t.tensor(y_test.values).float()

# Model

In [ ]:
# Define a model

model = nn.Sequential(nn.Linear(x_train.shape[1], 100), nn.ReLU(), nn.Linear(100,50), nn.ReLU(), nn.Linear(50,25), nn.ReLU(), nn.Linear(25, 1))

In [ ]:
# Define the loss function and the optimizer

loss_function = t.nn.MSELoss()
optimizer = t.optim.Adam(model.parameters(), lr=0.05) 

In [ ]:
print(x_train)
print(y_dev)
print(y_train)

tensor([[-1.0826e+00, -1.1189e+00, -1.5053e+00,  ..., -5.6287e-01,
         -2.8907e-02, -9.2232e-02],
        [-4.5963e-03, -1.1776e+00,  4.8098e-01,  ..., -4.4235e-01,
          1.5350e-02,  3.5314e-01],
        [-3.0845e-01, -1.0612e+00,  7.3594e-01,  ...,  1.0330e+00,
         -1.6869e-01,  7.7626e-01],
        ...,
        [ 5.1204e-01, -7.7624e-01, -1.1353e+00,  ...,  2.0932e+00,
          5.7537e+00,  3.6338e+00],
        [ 5.8048e-01,  9.7457e-01,  5.7132e-01,  ..., -2.4436e-01,
         -5.5204e-01, -4.3567e-02],
        [-7.0243e-01, -4.9564e-01,  1.2523e+00,  ..., -1.1171e+00,
          4.5657e-01,  2.8556e+00]])
tensor([1997., 2001., 2006.,  ..., 2001., 1991., 2005.])
tensor([2006., 1987., 1963.,  ..., 2008., 2008., 1976.])


In [ ]:
EPOCHS=3000
for i in range(EPOCHS):
  y_pred = model(x_train).squeeze()
  loss = loss_function(y_pred, y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if i%100 == 0:
    print(i, loss.item())

0 3993159.25
100 51213.63671875
200 1123.68603515625
300 223.20884704589844
400 137.2587432861328
500 111.54052734375
600 207.18006896972656
700 130.26210021972656
800 634.787109375
900 149.55386352539062
1000 91.93426513671875
1100 88.77936553955078
1200 119.93938446044922
1300 92.69599151611328
1400 83.3357162475586
1500 110.73416137695312
1600 306.3379821777344
1700 4533.544921875
1800 2343.12451171875
1900 580.8319702148438
2000 782.9967651367188
2100 171.38429260253906
2200 742.2029418945312
2300 1488.1324462890625
2400 1810.604248046875
2500 589.7868041992188
2600 279.13623046875
2700 1529.811279296875
2800 583.5845947265625
2900 10888.0498046875


In [ ]:
# we didn't use accuracy because it is not classification problem. Number of epochs increase the prediction

pred = model(x_test[0])
print("Ground truth:", y_test[0].item(), "Prediction:",pred.item())

Ground truth: 2005.0 Prediction: 2003.466552734375
